In [1]:
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from utils import *
import os

In [2]:
client=get_cluster()

In [3]:
dvs=['TSA','RH2M','QSOIL','QVEGE','QVEGT','RAIN','SNOW','GPP','NBP','SOILWATER_10CM','TWS','FSDS','TLAI','QRUNOFF']

In [21]:
dsws=[]
for yr in ['1850','2020','2090']:
    ds=get_ds(yr+'_era5_i04',dvs)
    la=ds.la*get_foco(ds)
    dsw=xr.Dataset()
    for v in ds.data_vars:
        if v!='la':
            dsw[v]=get_jas(ds[v],la)
    dsws.append(dsw)

In [22]:
ds=xr.concat(dsws,dim='year')
#

In [24]:
#read in the constrained circulation sw10cm data
sw0s=[]
for yr in [1850,2020,2090]:
    files=get_files(str(yr)+'_era5_i04',tape='h1')
    files=[f[0] for f in files]
    tmp=xr.open_dataset(files[0])
    la=get_foco(tmp)*tmp.area*tmp.landfrac
    def pp(ds):
        return gmean(ds.SOILWATER_10CM,la)
    sw0=xr.open_mfdataset(files,combine='nested',concat_dim='ens',preprocess=pp,parallel=True).isel(time=0)
    sw0s.append(sw0)

In [25]:
sw4=xr.concat(sw0s,dim='time')
sw4['year']=sw4['time.year']

In [26]:
ds['SW4']=da.swap_dims({'time':'year'})

In [28]:
ds.to_netcdf('/glade/derecho/scratch/djk2120/postp/jas/cce_era5_i04.nc')

### add some extra variables

In [4]:
fout='/glade/derecho/scratch/djk2120/postp/jas/cce_era5_i04.nc'
dsout=xr.open_dataset(fout)

In [5]:
dvs=['TREFHTMX']
yrs=[1850,2020,2090]
dsets=[get_ds(str(yr)+'_era5_i04',dvs,cmp='/atm/',tape='h1') for yr in yrs]
ds=xr.concat(dsets,dim='time')
la=np.cos(np.deg2rad(ds.lat))*get_foco(ds)
for v in dvs:
    ixt=(ds['time.month']>=7)&(ds['time.month']<=9)
    dsout[v]=gmean(ds[v].isel(time=ixt).groupby('time.year').mean(),la).T
os.remove(fout)
dsout.to_netcdf(fout)

In [5]:
dvs=['EFLX_LH_TOT','FSH']
yrs=[1850,2020,2090]
dsets=[get_ds(str(yr)+'_era5_i04',dvs) for yr in yrs]
ds=xr.concat(dsets,dim='time')
la=np.cos(np.deg2rad(ds.lat))*get_foco(ds)
for v in dvs:
    dsout[v]=get_jas(ds[v],la)
os.remove(fout)
dsout.to_netcdf(fout)

In [5]:
dvs=['PRECC','PRECL']
yrs=[1850,2020,2090]
dsets=[get_ds(str(yr)+'_era5_i04',dvs,cmp='/atm/') for yr in yrs]
ds=xr.concat(dsets,dim='time')
la=np.cos(np.deg2rad(ds.lat))*get_foco(ds)
for v in dvs:
    dsout[v]=get_jas(ds[v],la)
os.remove(fout)
dsout.to_netcdf(fout)

In [29]:
d='/glade/derecho/scratch/djk2120/postp/jas/'
v='TREFHTMX'
for yr in range(1850,2110,10):
    print(yr)
    f=d+v+'.'+str(yr)+'.nc'
    tmp=xr.open_dataset(f)
    tmp[v]=tmp[v].T
    os.remove(f)
    tmp.to_netcdf(f)

1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
